In [1]:
import pandas as pd
import numpy as np
import datetime

### 📍 Chargement des données :

In [2]:
df_full = pd.read_csv("df_full_entretien.csv")
#df_full = df_full.drop(columns=["Unnamed: 0"])

df_cluster_advanced = pd.read_csv("df_cluster_advanced.csv")
df_cluster_advanced_scaled = pd.read_csv("df_cluster_advanced_scaled.csv")

---

### 📍 Préparation des données :

In [3]:
df_cluster_advanced_scaled = df_cluster_advanced_scaled.drop(columns=["Unnamed: 0"])
df_cluster_advanced_scaled

,review_score,CA_tot_client,Ancienneté,fréq_achat,panier_moyen,cluster_KMEANS
0,0.678325,-0.391214,0.689238,0.866507,-0.349911,0
1,-0.064470,0.479467,1.596773,0.023994,0.008972,0
2,0.678325,0.237951,-0.108095,0.069360,0.395082,1
3,-0.064470,-0.597387,-1.443467,-1.265699,-0.594041,1
4,0.678325,0.241992,2.121846,2.298778,0.399867,0
...,...,...,...,...,...,...
98668,0.678325,0.691479,-0.749852,-0.572246,0.932104,1
98669,0.678325,0.916065,-1.281408,-1.103677,1.198036,1
98670,0.678325,-0.207090,0.429943,0.607272,-0.131891,0
98671,0.678325,-0.404282,0.877228,1.054452,-0.365385,0


In [4]:
df_cluster_advanced = df_cluster_advanced.set_index("customer_unique_id")
df_cluster_advanced_scaled.index = df_cluster_advanced.index
df_cluster_advanced

,review_score,CA_tot_client,Ancienneté,fréq_achat,panier_moyen
customer_unique_id,,,,,
871766c5855e863f6eccc05f988b23cb,5.0,58.90,400.0,400.0,58.90
eb28e67c4c0b83846050ddfb8a35d051,4.0,252.78,540.0,270.0,126.39
3818d81c6709e39d06b2738a8d3a2474,5.0,199.00,277.0,277.0,199.00
af861d436cfc08b2c2ddefd0ba074622,4.0,12.99,71.0,71.0,12.99
64b576fb70d441e8f1b2d7d446e483c5,5.0,199.90,621.0,621.0,199.90
...,...,...,...,...,...
0c9aeda10a71f369396d0c04dce13a64,5.0,299.99,178.0,178.0,299.99
0da9fe112eae0c74d3ba1fe16de0988b,5.0,350.00,96.0,96.0,350.00
cd79b407828f02fdbba457111c38e4c4,5.0,99.90,360.0,360.0,99.90


In [5]:
#df_cluster_advanced = df_cluster_advanced.drop(columns=["cluster_KMEANS"])
df_cluster_advanced_scaled = df_cluster_advanced_scaled.drop(columns=["cluster_KMEANS"])

### 📍  Plage des dates à évaluer :

In [6]:
df_full.order_purchase_timestamp.min()

'2016-09-04 21:15:19'

In [7]:
df_full.order_purchase_timestamp.max()

'2018-10-17 17:30:18'

---

# 👩‍🚀👨‍🚀 Test de durée optimale d'entretien des systèmes de clustering

---

##### Le premier client a 773 jours d'ancienneté.
##### Le dernier client 45 jours.
##### Les premiers tests d'entraînement du système de clustering réalisés ont donc porté sur une période de 1 an à compter de l'obtention du premier client soit ancienneté entre 773 jours et 408 jours d'ancienneté afin d'étalonner notre algorithme sur la première année d'activité.

#### Les deux premiers mois d'activité ont été particulièrement spécifiques raisons pour laquelle notre test suivant nous a conduit à les retirer de l'entraînement initial de notre sustème de clustering. (plage de d'ancienneté pour l'entraînement : 714,347)

In [8]:
773-517

256

In [9]:
773-408

365

#### 📌 Préparation DataFrame :

In [10]:
#df_RFM['Ancienneté'] = df_RFM['Ancienneté'].apply(lambda x: int(x.split()[0]))

##### Comme découvert dans les pré-tests le KMEANS avec 3 clusters est idéal afin de traiter notre problématique de clustering.

### 📍 1 -  Attribuer un cluster à l'ensemble des customer_id_unique et sur la totalité de la durée analysable :

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#scaler = StandardScaler()
scaled_data = df_cluster_advanced_scaled

---

### 📍 Entraîner notre KMEANS sur tout le jeu de données :

In [12]:
km = KMeans(n_clusters=5, init='k-means++', n_init = 10)
km.fit(df_cluster_advanced_scaled)
res = silhouette_score(df_cluster_advanced_scaled, km.labels_)
print(res)

0.3238785563636519


---

### 📍 Intégrer au jeu de données les clusters sur une année :

In [13]:
scaled_data = pd.DataFrame(df_cluster_advanced_scaled, columns = df_cluster_advanced.columns)
df_RFM = df_cluster_advanced_scaled
df_RFM_cluster = df_cluster_advanced
df_RFM_cluster["cluster"] = km.labels_

scaled_data_with_clusters = df_cluster_advanced_scaled
scaled_data_with_clusters["cluster"] = km.labels_

---

---

# 📍 Test de longévité du système entraîné :

---

### 👩‍💻 Test avec calcul de l'ARI sur les nouvelles données uniquement :
### Entraînement sur 1 an et vérification de stabilité :

---

In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Nombre d'itérations
n_iterations = 25

# Entraînement modèle de base :
df_RFM_1_an = df_cluster_advanced
df_RFM_1_an = df_RFM_1_an.loc[(df_RFM_1_an["Ancienneté"] < 773) &
                              (df_RFM_1_an["Ancienneté"] > 408), :]

scaler_init = StandardScaler()
scaled_data_init = scaler_init.fit_transform(df_RFM_1_an)
scaled_data_init = pd.DataFrame(scaled_data_init, columns = df_RFM_1_an.columns)

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=1)
kmeans.fit(scaled_data_init)

# Effectuer plusieurs K-means avec des initialisations aléatoires
for i in range(n_iterations):
    i += 1
    # Étiquettes de classe réelles (pour l'évaluation de l'ARI)
    y_true = df_RFM_cluster.loc[(df_RFM_cluster["Ancienneté"] < 408) &
                                (df_RFM_cluster["Ancienneté"] > (408-(i*15))),
                                'cluster' ]
    
    X = df_cluster_advanced.loc[(df_cluster_advanced["Ancienneté"] < 408) &
                                (df_cluster_advanced["Ancienneté"] > (408-(i*15))), :]
    
    #scaler = StandardScaler()
    X_scaled = scaler_init.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns = df_cluster_advanced.columns)
    
    # Effectuer le clustering
    labels = kmeans.predict(X_scaled)
    
    # Obtenir les étiquettes de cluster assignées à chaque échantillon
    #labels = kmeans.labels_
    
    # Calculer l'Adjusted Rand Index (ARI)
    ari = adjusted_rand_score(y_true, labels)
    
    # Afficher les résultats
    print(f"Itération {i}:")
    print("+",i*15,'jours')
    print("Nombre de données exploitées : ", X.shape)
    print("Étiquettes des échantillons :", labels)
    print("Adjusted Rand Index (ARI) :", ari)
    print()

C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Itération 1:
+ 15 jours
Nombre de données exploitées :  (2120, 6)
Étiquettes des échantillons : [3 3 0 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.9106367406102999

Itération 2:
+ 30 jours
Nombre de données exploitées :  (4237, 6)
Étiquettes des échantillons : [3 3 3 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.91596427093984

Itération 3:
+ 45 jours
Nombre de données exploitées :  (6556, 6)
Étiquettes des échantillons : [3 3 3 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.8980396088395357

Itération 4:
+ 60 jours
Nombre de données exploitées :  (8632, 6)
Étiquettes des échantillons : [3 3 0 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.8845914782967699

Itération 5:
+ 75 jours
Nombre de données exploitées :  (11253, 6)
Étiquettes des échantillons : [3 3 0 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.8767066353283792

Itération 6:
+ 90 jours
Nombre de données exploitées :  (16391, 6)
Étiquettes des échantillons : [3 3 3 ... 3 3 3]
Adjusted Rand Index (ARI) : 0.8634866523618474

Itération 7:
+ 105 jours
Nom

In [15]:
df_RFM_cluster.to_csv("fichier_client_avec_clusters.csv")

---

## 👨‍🚀👩‍🚀🚀 Nécessité de suivi au bout de 135 à 150 jours soit au bout de 5 mois. Le modèle sera surement plus robuste à l'avenir mais nous avons besoin de plus de données donc de temps d'activité pour diminuer le besoin de suivi.

---
---
---

## Modèle qui pourra être étudié lors du suivi à +4mois et demi ou +6 mois.
## Classification automatisée auto-apprenante tous les quinze jours  ! ! ! ! (Brouillon)

---

In [24]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_cluster_advanced)
scaled_data = pd.DataFrame(scaled_data, columns = df_cluster_advanced.columns)

n_iterations = 25

# Effectuer plusieurs K-means avec des initialisations aléatoires
for i in range(n_iterations):
    
    # Étiquettes de classe réelles (pour l'évaluation de l'ARI)
    y_true = df_RFM_cluster.loc[(df_RFM_cluster["Ancienneté"] < 773) &
                            (df_RFM_cluster["Ancienneté"] > (408-i*15)), 'cluster' ]
    
    X = df_cluster_advanced.loc[(df_cluster_advanced["Ancienneté"] < 773) &
                                (df_cluster_advanced["Ancienneté"] > (408-(i*15))), :]
    
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = df_cluster_advanced.columns)
    #X_scaled = pd.DataFrame(X_scaled, columns = df_cluster_advanced.columns)
    
    # Créer un objet KMeans avec 5 clusters et une init aléatoire
    kmeans = KMeans(n_clusters=5, init='k-means++', random_state=1)
    
    # Effectuer le clustering
    kmeans.fit(X_scaled)
    
    # Obtenir les étiquettes de cluster assignées à chaque échantillon
    labels = kmeans.labels_
    
    # Calculer l'Adjusted Rand Index (ARI)
    ari = adjusted_rand_score(y_true, labels)
    
    # Afficher les résultats
    print(f"Itération {i}:")
    print("+",i*15,'jours')
    print("Nombre de données exploitées : ", X.shape)
    print("Étiquettes des échantillons :", labels)
    print("Adjusted Rand Index (ARI) :", ari)
    print()

C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Itération 0:
+ 0 jours
Nombre de données exploitées :  (24248, 6)
Étiquettes des échantillons : [3 4 1 ... 2 0 1]
Adjusted Rand Index (ARI) : 0.37267920263051285



C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Itération 1:
+ 15 jours
Nombre de données exploitées :  (26527, 6)
Étiquettes des échantillons : [0 0 3 ... 4 2 1]
Adjusted Rand Index (ARI) : 0.45252745212907686



C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Itération 22:
+ 330 jours
Nombre de données exploitées :  (92234, 6)
Étiquettes des échantillons : [2 2 2 ... 2 1 0]
Adjusted Rand Index (ARI) : 1.0



C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Itération 23:
+ 345 jours
Nombre de données exploitées :  (96456, 6)
Étiquettes des échantillons : [3 3 3 ... 3 1 0]
Adjusted Rand Index (ARI) : 1.0



C:\Users\cyril\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Itération 24:
+ 360 jours
Nombre de données exploitées :  (98665, 6)
Étiquettes des échantillons : [0 0 0 ... 0 2 1]
Adjusted Rand Index (ARI) : 1.0



In [25]:
# On pourra aussi tester ce genre de modèle à rentraînement automatique lorsque nous seront moins affecté par la spécificité
# des saisons et que nous aurons plus de données temporelles (plus de temps d'activité).